## RAGAS Evaluation Metrics for RAG Pipelines

When you build a RAG (Retrieval-Augmented Generation) system, you want to measure **how well** it retrieves and generates relevant, factual answers. RAGAS provides industry-standard metrics to do this.

---

### **1. LLM Context Recall**
**What it measures:**  
Checks if the AI's final answer actually uses the retrieved context chunks. High recall means the answer uses as much relevant retrieved information as possible.

**Why it matters:**  
Good recall shows your retriever is surfacing useful facts that the LLM then uses — important for trust and explainability.

Example: If you ask "When was Python created?" and the retrieved context says "Python was created in 1991 by Guido van Rossum." — the LLM output should reflect both 1991 and Guido. If it only mentions 1991, recall is lower.

---

### **2. Context Precision**
**What it measures:**  
Measures how much of the retrieved context was actually useful for answering the question. High precision means you’re not pulling in lots of irrelevant text.

**Why it matters:**  
High precision = more efficient retrieval = faster & cheaper pipelines. Less noise, more signal!

Example: If you ask about the Moon, but the retrieved docs also mention Mars missions, your context precision will drop.

---

### ✅ **3. Faithfulness**
**What it measures:**  
Checks if the AI’s generated answer stays true to the context. It penalizes hallucinations or made-up information.

**Why it matters:**  
Faithfulness is critical for factual correctness in production. No one wants a chatbot hallucinating numbers or fake facts!

Example: If context says "The Moon has no atmosphere" but the answer says "The Moon has a thin oxygen atmosphere," that’s unfaithful.

---

### ✅ **4. Answer Relevancy**
**What it measures:**  
Evaluates whether the answer actually addresses the user’s question.

**Why it matters:**  
You could have perfect facts, but if your answer is off-topic or incomplete, it’s useless for the user.

Example: If you ask "What is the Moon made of?" and it replies with "The Moon orbits Earth," that’s irrelevant.

---

### ✅ **5. Factual Correctness**
**What it measures:**  
Compares the generated answer to a ground truth reference answer (if available). It ensures the answer is factually correct and matches trusted knowledge.

**Why it matters:**  
This is the gold standard — especially in legal, medical, or academic use cases.

Example: If you ask "Who discovered America?" and it says "Abraham Lincoln," factual correctness is 0!

---

**Together**, these metrics help you debug and improve your retriever, reranker, and generator stages for a high-quality RAG pipeline.

In [1]:
# Install Required Dependencies
!pip install -q langchain sentence-transformers faiss-cpu pypdf openai langchain-community langchain-openai ragas datasets pandas numpy langchain-huggingface
!pip install -q --upgrade ragas

print("All dependencies installed successfully.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 835.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211

In [83]:
# Import Required Libraries
import os, time, json, torch, pandas as pd, numpy as np
from typing import List
import warnings
warnings.filterwarnings("ignore")

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import RetrievalQA

from sentence_transformers import CrossEncoder
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision, FactualCorrectness
from datasets import Dataset

print("All libraries imported. PyTorch version:", torch.__version__)

All libraries imported. PyTorch version: 2.6.0+cu124


In [3]:
# Configuration Setup
class RAGConfig:
    CHUNK_SIZE = 500
    CHUNK_OVERLAP = 50
    EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
    LLM_MODEL = "gpt-4"  # or "gpt-3.5-turbo"
    RERANKER_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"
    RETRIEVER_K = 15
    RETRIEVER_FETCH_K = 30
    TOP_K_AFTER_RERANK = 3

config = RAGConfig()
print("Configuration initialized.")

Configuration initialized.


In [5]:
# Load and Process PDF
PDF_PATH = "/content/solid-python.pdf"

def load_and_process_documents(pdf_path, config):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load_and_split()
    splitter = CharacterTextSplitter(chunk_size=config.CHUNK_SIZE, chunk_overlap=config.CHUNK_OVERLAP)
    return splitter.split_documents(docs)

documents = load_and_process_documents(PDF_PATH, config)
print(f"Loaded {len(documents)} document chunks.")

Loaded 22 document chunks.


In [6]:
# Setup Embeddings and Vector Store
def setup_vectorstore(docs, config):
    embed = HuggingFaceEmbeddings(model_name=config.EMBEDDING_MODEL, model_kwargs={'device':'cpu'})
    store = FAISS.from_documents(docs, embed)
    return store, embed

vectorstore, embedding_model = setup_vectorstore(documents, config)
print("Vector store is ready.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store is ready.


In [15]:
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY_')
# Setup OpenAI LLM
def setup_llm(config):
    key = os.getenv("OPENAI_API_KEY")
    if not key:
        raise ValueError("Set your OPENAI_API_KEY in environment variables.")
    return ChatOpenAI(model_name=config.LLM_MODEL, openai_api_key=key, temperature=0.1, max_tokens=1024)

llm = setup_llm(config)
print("OpenAI LLM initialized.")

OpenAI LLM initialized.


In [16]:
# Reranked Retriever
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document
from typing import List, Any
# remove dataclass and field from dataclasses
from pydantic import Field # Import Field

class RerankedRetriever(BaseRetriever):
    vectorstore: Any = Field(...)
    reranker: Any = Field(...)
    top_k: int = Field(default=3)
    search_kwargs: dict = Field(default_factory=lambda: {"k": config.RETRIEVER_K, "fetch_k": config.RETRIEVER_FETCH_K})

    def _get_relevant_documents(self, query: str, **kwargs) -> List[Document]:
        init_ret = self.vectorstore.as_retriever(search_type="mmr", search_kwargs=self.search_kwargs)
        docs = init_ret.get_relevant_documents(query)
        if not docs:
            return []
        pairs = [[query, d.page_content] for d in docs]
        scores = self.reranker.predict(pairs)
        ranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
        return [d for d, _ in ranked[:self.top_k]]

def setup_reranker_and_retriever(vectorstore, config):
    reranker = CrossEncoder(config.RERANKER_MODEL)
    retriever = RerankedRetriever(vectorstore=vectorstore, reranker=reranker, top_k=config.TOP_K_AFTER_RERANK,
            search_kwargs={"k": config.RETRIEVER_K, "fetch_k": config.RETRIEVER_FETCH_K})
    return retriever, reranker

retriever, reranker = setup_reranker_and_retriever(vectorstore, config)
print("Retriever + Reranker ready.")

Retriever + Reranker ready.


In [17]:
# Create QA Chain
def create_qa_chain(llm, retriever):
    prompt = ChatPromptTemplate.from_template(
        "You are a helpful assistant. Use the context to answer the question.\n"
        "If the answer is not in the context, say you don't know.\n\n"
        "Context: {context}\nQuestion: {question}\nAnswer:"
    )
    return RetrievalQA.from_chain_type(
        llm=llm, chain_type="stuff", retriever=retriever,
        chain_type_kwargs={"prompt":prompt}, return_source_documents=True
    )

qa_chain = create_qa_chain(llm, retriever)
print("QA chain is set up.")

QA chain is set up.


In [18]:
# Test a Sample Question
def test_single_query(chain, q):
    res = chain({"query": q})
    print("Answer:", res["result"])
    print("Sources:", [doc.metadata.get("page","N/A") for doc in res["source_documents"]])
    return res

test_single_query(qa_chain, "What is the main objective of the document?")

Answer: The main objective of the document is to outline the aims for keeping a software application flexible, robust, reusable, and developable, to explain the five aspects of a class, and to describe the five principles for SOLID software design.
Sources: [3, 18, 19]


{'query': 'What is the main objective of the document?',
 'result': 'The main objective of the document is to outline the aims for keeping a software application flexible, robust, reusable, and developable, to explain the five aspects of a class, and to describe the five principles for SOLID software design.',
 'source_documents': [Document(id='c009c554-c0dc-42df-9443-8208ccbdefc4', metadata={'producer': 'LaTeX with hyperref and thumbpdf', 'creator': 'okular', 'creationdate': '2024-02-22T19:02:22+00:00', 'author': 'Jan Wedekind', 'title': 'The SOLID Principles', 'subject': 'After Catchup Presentation 22nd Feb 2022', 'keywords': 'solid, software, design', 'moddate': '2024-02-22T19:02:22+00:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022/Debian) kpathsea version 6.3.4', 'source': '/content/solid-python.pdf', 'total_pages': 22, 'page': 3, 'page_label': '4'}, page_content='Aims\nThursday, Feb 22nd 2024 4/22\nIn contrast we want t

In [19]:
# Prepare Evaluation Dataset
def prepare_ragas_dataset(chain, questions, ground_truths=None):
    items = []
    for i, q in enumerate(questions):
        res = chain({"query": q})
        item = {"question": q, "answer": res["result"],
                "contexts": [d.page_content for d in res["source_documents"]]}
        if ground_truths and i < len(ground_truths):
            item["ground_truth"] = ground_truths[i]
        items.append(item)
    return Dataset.from_list(items)

evaluation_questions = [
    "What is the main objective of the document?",
    "What are the key principles discussed in the document?",
    "How does the document define solid principles?"
]
ground_truths = ["The document explains SOLID principles in Python."]

eval_dataset = prepare_ragas_dataset(qa_chain, evaluation_questions, ground_truths)
print(f"Prepared {len(eval_dataset)} evaluation entries.")

Prepared 3 evaluation entries.


In [85]:
# Run RAGAS Evaluation
def run_ragas_evaluation(dataset, llm):
  metrics = [FactualCorrectness(), faithfulness, answer_relevancy, context_precision, context_recall]
  return evaluate(dataset=dataset, metrics=metrics, llm=llm)

ragas_results = run_ragas_evaluation(eval_dataset, llm)
print("RAGAS evaluation complete.")

Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[5]: AssertionError(Reference is not set)
ERROR:ragas.executor:Exception raised in Job[8]: KeyError('reference')
ERROR:ragas.executor:Exception raised in Job[9]: KeyError('reference')
ERROR:ragas.executor:Exception raised in Job[10]: AssertionError(Reference is not set)
ERROR:ragas.executor:Exception raised in Job[13]: KeyError('reference')
ERROR:ragas.executor:Exception raised in Job[14]: KeyError('reference')


RAGAS evaluation complete.


In [86]:
type(ragas_results), ragas_results

(ragas.dataset_schema.EvaluationResult,
 {'factual_correctness(mode=f1)': 0.2500, 'faithfulness': 1.0000, 'answer_relevancy': 0.9845, 'context_precision': 0.3333, 'context_recall': 0.0000})

In [88]:
# Display Evaluation Results
def display_ragas_results(results):
    print("Evaluation Results:")
    res={}
    matric=['factual_correctness(mode=f1)','faithfulness', 'answer_relevancy', 'context_precision','context_recall']
    for sc in matric:
        print(f"{sc}: {results[sc][2]:.4f} ({results[sc][0]*100:.1f}%), Type: {type(results[sc][2])}")
        res[sc]=float(results[sc][2]*100) if np.float64(results[sc][0]*100)>100.00 else float(results[sc][0]*100)
    return res

result_dat=display_ragas_results(ragas_results)

Evaluation Results:
factual_correctness(mode=f1): nan (25.0%), Type: <class 'float'>
faithfulness: 1.0000 (100.0%), Type: <class 'float'>
answer_relevancy: 0.9536 (100.0%), Type: <class 'numpy.float64'>
context_precision: nan (33.3%), Type: <class 'float'>
context_recall: nan (0.0%), Type: <class 'float'>


In [91]:
result_dat

{'factual_correctness(mode=f1)': 25.0,
 'faithfulness': 100.0,
 'answer_relevancy': 95.36159575012654,
 'context_precision': 33.333333329999995,
 'context_recall': 0.0}

In [89]:
# Analyze and Recommend
def analyze_performance(results):
    avg = np.mean([v for v in results.values() if isinstance(v, float)])
    print(f"Average Score: {avg:.3f}")
    if avg >= 0.8:
        print("Excellent performance.")
    elif avg >= 0.6:
        print("Good performance. Room for improvement.")
    else:
        print("Needs improvement. Consider optimizing models or data.")

analyze_performance(result_dat)

Average Score: 50.739
Excellent performance.


In [90]:
# Save Results and Generate Report
import json

def save_results_and_generate_report(results, test_questions, config):
    report_data = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "configuration": {
            "chunk_size": config.CHUNK_SIZE,
            "embedding_model": config.EMBEDDING_MODEL,
            "llm_model": config.LLM_MODEL,
            "retriever_k": config.RETRIEVER_K
        },
        "evaluation_metrics": {k: float(v) for k, v in results.items() if isinstance(v, float)},
        "test_questions": test_questions
    }

    with open("rag_evaluation_report.json", "w") as f:
        json.dump(report_data, f, indent=2)

    pd.DataFrame([report_data["evaluation_metrics"]]).to_csv("rag_metrics.csv", index=False)

    with open("rag_evaluation_report.md", "w") as f:
        f.write("# RAG Evaluation Report\n\n")
        for k, v in report_data["evaluation_metrics"].items():
            f.write(f"- {k}: {v:.4f}\n")

    print("Report saved.")

save_results_and_generate_report(result_dat, evaluation_questions, config)

Report saved.
